<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Imports-and-Update-The-Latest-Data" data-toc-modified-id="Imports-and-Update-The-Latest-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports and Update The Latest Data</a></span></li><li><span><a href="#Historical-air-4-Thai-Data" data-toc-modified-id="Historical-air-4-Thai-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Historical air 4 Thai Data</a></span></li><li><span><a href="#Power-Plants" data-toc-modified-id="Power-Plants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Power Plants</a></span></li></ul></div>

# Data Source 
1. Berekely Earth 'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/'
2. Screaped Air4Thai Data 'http://air4thai.pcd.go.th/webV2/history/'
3. CDC Data 'https://www.cmuccdc.org/download_json/'
4. Old Air4Thai data from Thailand EPA

# Imports and Update The Latest Data

In [1]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *

In [2]:
b_folder='../data/pm25/'
a4th_folder='../data/air4thai_hourly/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'

In [3]:
# Data from Berkeley Earth Projects: 
download_b_data(data_folder='../data/pm25/', url='http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/')
get_city_info(data_folder='../data/pm25/')

100% [............................................................................] 796692 / 796692

In [4]:
download_cdc_data(station_url='https://www.cmuccdc.org/api/ccdc/stations', 
                  dl_url= 'https://www.cmuccdc.org/download_json/', 
                  data_folder='../data/cdc_data/')

  0%|                                                                                          | 0/404 [00:00<?, ?it/s]

number of stations 404


100%|████████████████████████████████████████████████████████████████████████████████| 404/404 [20:52<00:00,  3.10s/it]


In [5]:
update_last_air4Thai(url='http://air4thai.pcd.go.th/webV2/history/',data_folder='../data/air4thai_hourly/')

0it [00:00, ?it/s]

['02t', '03t', '05t', '08t', '10t', '11t', '12t', '13t', '14t', '16t', '17t', '18t', '19t', '20t', '21t', '22t', '24t', '25t', '26t', '27t', '28t', '29t', '30t', '31t', '32t', '33t', '34t', '35t', '36t', '37t', '38t', '39t', '40t', '41t', '42t', '43t', '44t', '46t', '47t', '50t', '52t', '53t', '54t', '57t', '58t', '59t', '60t', '61t', '62t', '63t', '67t', '68t', '69t', '70t', '71t', '72t', '73t', '74t', '75t', '76t', '77t', '79t', '80t', '81t', '82t', '83t', '84t', 'm1', 'm4', 'm8', 'm9', 'o10', 'o20', 'o22', 'o23', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29']


81it [27:04, 20.05s/it]


# Historical air 4 Thai Data

In [6]:
# load stations information
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

# find stations in Chiangmai and parase that files
cm_station_ids = []
for i, stations in enumerate(station_info):
    if 'Chiang Mai' in stations['areaEN']:
        # ignore station that start with o
        if 'o' not in stations['stationID']:
            cm_station_ids.append(stations['stationID'])
print(cm_station_ids)

['35t', '36t', 'm9']


In [22]:
# parase historical data 
for station_id in cm_station_ids:
    # find all files that start with this stations
    p = Path(aqm_folder)
    filenames = []
    for i in p.glob('**/*.xlsx'):
        if station_id in i.name:
            filenames.append(str(i))
        
    # if filename exist load that file
    if len(filenames) >0:

        save_filename = aqm_folder + 'process/' + station_id + '.csv'
        print('save filename', save_filename)
        station_data = read_his_xl(filenames[0])
        #print(station_data.head())

        # save the data if the dataframe is not empty
        if len(station_data)> 0:
            station_data.to_csv(save_filename,index=True)

save filename ../data/aqm_hourly2/process/35t.csv
save filename ../data/aqm_hourly2/process/36t.csv


In [45]:
cm_data1 = pd.read_csv(aqm_folder + 'process/35t.csv')
cm_data1['datetime'] = pd.to_datetime(cm_data1['datetime'])
cm_data1 = cm_data1.set_index('datetime')
# keep only gas columns
cm_data1 = cm_data1[['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']]

cm_data2 = pd.read_csv(aqm_folder + 'process/36t.csv')
cm_data2['datetime'] = pd.to_datetime(cm_data2['datetime'])
cm_data2 = cm_data2.set_index('datetime')


# Power Plants

In [92]:
url = 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B9%82%E0%B8%A3%E0%B8%87%E0%B9%84%E0%B8%9F%E0%B8%9F%E0%B9%89%E0%B8%B2%E0%B9%83%E0%B8%99%E0%B9%84%E0%B8%97%E0%B8%A2'
url = 'https://en.wikipedia.org/wiki/List_of_power_stations_in_Thailand'
table_list = pd.read_html(url)
len(table_list)
p_folder = '../data/power_plants/'
for i, table in enumerate(table_list):
    table.to_csv(p_folder + f'table_eng{i}.csv',index=False)